# Capstone: Financial Research Copilot

## Step 0: Setup and Configuration
Importing necessary libraries (yfinance, pandas, google-generativeai) and configuring the Google Gemini client using the API key from `.env`.

In [1]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import google.generativeai as genai
import json
from ddgs import DDGS

c:\Users\yuhes\Courses\Google & Kaggle Gen AI\Capstone\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Load environment variables
load_dotenv()

# 2. Get API Key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("Error: GOOGLE_API_KEY not found in .env file.")
else:
    print("API Key loaded successfully.")
    
    # 3. Configure the client
    try:
        genai.configure(api_key=api_key)
        print("Google GenAI Client configured successfully.")
    except Exception as e:
        print(f"Error configuring client: {e}")

API Key loaded successfully.
Google GenAI Client configured successfully.


## Step 1.1: Price History Tool
Defining `get_asset_price_history` to fetch OHLCV data for Stocks and ETFs using `yfinance`. This tool returns structured JSON data that the agent can analyze.


In [3]:
from typing import Dict, Any

def get_asset_price_history(symbol: str, asset_type: str = "equity", period: str = "1mo") -> Dict[str, Any]:
    """
    Fetch historical OHLCV price data for an asset.
    
    Args:
        symbol (str): Ticker symbol (e.g., 'AAPL', 'BTC-USD').
        asset_type (str): Type of asset ('equity', 'crypto', 'etf'). Defaults to 'equity'.
        period (str): Data period to download (e.g., '1mo', '3mo', '1y'). Defaults to '1mo'.
        
    Returns:
        dict: Structured data containing status and price history.
    """
    try:
        # Handle crypto symbols for yfinance (usually needs -USD suffix if not provided)
        if asset_type.lower() == 'crypto' and not symbol.endswith('-USD'):
            symbol = f"{symbol}-USD"
            
        ticker = yf.Ticker(symbol)
        
        # Fetch history
        hist = ticker.history(period=period)
        
        if hist.empty:
            return {
                "status": "error",
                "message": f"No data found for {symbol}. Check ticker or period."
            }
            
        # Format data for the agent (convert to list of dicts)
        # We limit to the last 60 records to keep context small if period is long
        data_records = []
        for date, row in hist.tail(60).iterrows():
            data_records.append({
                "date": date.strftime('%Y-%m-%d'),
                "close": round(row['Close'], 2),
                "volume": int(row['Volume'])
            })
            
        return {
            "status": "success",
            "symbol": symbol,
            "currency": ticker.info.get('currency', 'USD'),
            "data": data_records
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [4]:
# --- Testing the tool  ---
print("Testing Tool 1.1 with AAPL...")
result = get_asset_price_history("AAPL", period="5d")
result = json.dumps(result, indent=4)
print(result)


Testing Tool 1.1 with AAPL...
{
    "status": "success",
    "symbol": "AAPL",
    "currency": "USD",
    "data": [
        {
            "date": "2025-11-21",
            "close": 271.49,
            "volume": 59030800
        },
        {
            "date": "2025-11-24",
            "close": 275.92,
            "volume": 65585800
        },
        {
            "date": "2025-11-25",
            "close": 276.97,
            "volume": 46914200
        },
        {
            "date": "2025-11-26",
            "close": 277.55,
            "volume": 33431400
        },
        {
            "date": "2025-11-28",
            "close": 278.85,
            "volume": 20135600
        }
    ]
}


## Step 1.2: Fundamentals Tool
Defining `get_asset_fundamentals` to retrieve key financial metrics (P/E, Market Cap, Sector) using `yfinance`. This helps the agent assess value and risk.


In [5]:
def get_asset_fundamentals(symbol: str, asset_type: str = "equity") -> Dict[str, Any]:
    """
    Fetch fundamental data (P/E, Market Cap, Sector, Summary) for an asset.
    
    Args:
        symbol (str): Ticker symbol (e.g., 'AAPL').
        asset_type (str): 'equity' or 'etf'. (Crypto has limited fundamentals in yf).
        
    Returns:
        dict: Key fundamentals or error message.
    """
    try:
        if asset_type.lower() == 'crypto' and not symbol.endswith('-USD'):
             symbol = f"{symbol}-USD"

        ticker = yf.Ticker(symbol)
        info = ticker.info
        
        # Extract only the most relevant fields to save context tokens
        fundamentals = {
            "symbol": symbol,
            "name": info.get("shortName", "Unknown"),
            "sector": info.get("sector", "Unknown"),
            "industry": info.get("industry", "Unknown"),
            "market_cap": info.get("marketCap", "N/A"),
            "trailing_pe": info.get("trailingPE", "N/A"),
            "forward_pe": info.get("forwardPE", "N/A"),
            "dividend_yield": info.get("dividendYield", "N/A"),
            "fifty_two_week_high": info.get("fiftyTwoWeekHigh", "N/A"),
            "fifty_two_week_low": info.get("fiftyTwoWeekLow", "N/A"),
            "summary": info.get("longBusinessSummary", "No summary available")[:300] + "..." # Truncate summary
        }
        
        return {
            "status": "success",
            "data": fundamentals
        }
        
    except Exception as e:
        return {"status": "error", "message": str(e)}


In [6]:
# --- Test the tool immediately ---
print("Testing Tool 1.2 with MSFT...")
fund_result = get_asset_fundamentals("MSFT")
fund_result = json.dumps(fund_result, indent=4)
print(fund_result)

Testing Tool 1.2 with MSFT...
{
    "status": "success",
    "data": {
        "symbol": "MSFT",
        "name": "Microsoft Corporation",
        "sector": "Technology",
        "industry": "Software - Infrastructure",
        "market_cap": 3657192177664,
        "trailing_pe": 34.96873,
        "forward_pe": 32.91037,
        "dividend_yield": 0.74,
        "fifty_two_week_high": 555.45,
        "fifty_two_week_low": 344.79,
        "summary": "Microsoft Corporation develops and supports software, services, devices, and solutions worldwide. The company's Productivity and Business Processes segment offers Microsoft 365 Commercial, Enterprise Mobility + Security, Windows Commercial, Power BI, Exchange, SharePoint, Microsoft Teams, Security a..."
    }
}


## Step 1.3: Technical Analysis Tool
Defining `calculate_technical_indicators` to compute Volatility (Standard Deviation) and Moving Averages. This gives the agent quantitative data to assess risk.


In [7]:
def calculate_technical_indicators(symbol: str, period: str = "3mo") -> Dict[str, Any]:
    """
    Calculate technical indicators (Volatility, SMA) for a symbol.
    
    Args:
        symbol (str): Ticker symbol.
        period (str): Data lookback period (default '3mo').
        
    Returns:
        dict: Calculated metrics including annualized volatility and current trend.
    """
    try:
        # We reuse our first tool logic to get the raw dataframe usually, 
        # but calling yfinance directly here is cleaner for the standalone tool.
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period=period)
        
        if hist.empty:
             return {"status": "error", "message": "No data for calculations."}
        
        # 1. Calculate Daily Returns
        hist['Returns'] = hist['Close'].pct_change()
        
        # 2. Annualized Volatility (Standard Deviation of returns * sqrt(252 trading days))
        volatility = hist['Returns'].std() * np.sqrt(252)
        
        # 3. Simple Moving Average (50-day) - requiring enough data
        if len(hist) >= 50:
            sma_50 = hist['Close'].rolling(window=50).mean().iloc[-1]
        else:
            sma_50 = "Not enough data"
            
        current_price = hist['Close'].iloc[-1]
        
        # Determine trend
        trend = "Neutral"
        if isinstance(sma_50, (int, float)):
            if current_price > sma_50:
                trend = "Uptrend (Above 50d SMA)"
            else:
                trend = "Downtrend (Below 50d SMA)"
        
        return {
            "status": "success",
            "symbol": symbol,
            "current_price": round(current_price, 2),
            "annualized_volatility": round(volatility * 100, 2), # as percentage
            "sma_50": round(sma_50, 2) if isinstance(sma_50, float) else sma_50,
            "trend": trend
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}


In [8]:
# --- Test the tool ---
print("Testing Tool 1.3 with NVDA...")
tech_result = calculate_technical_indicators("NVDA")
tech_result = json.dumps(tech_result, indent=4)
print(tech_result)

Testing Tool 1.3 with NVDA...
{
    "status": "success",
    "symbol": "NVDA",
    "current_price": 177.0,
    "annualized_volatility": 37.41,
    "sma_50": 186.83,
    "trend": "Downtrend (Below 50d SMA)"
}


## Step 1.4: Market Opportunity Scanner Tool
Defining `scan_market_opportunities` to filter a watchlist of major assets for potential "Buys". It looks for:
1. **Value**: Low P/E ratio (< 25).
2. **Momentum**: Strong recent performance.
3. **Dip Buys**: Assets down significantly from their 52-week high.


In [9]:
def search_market_trends(query: str) -> Dict[str, Any]:
    """
    Search the web for financial news, trending stocks, or market analysis.
    Use this to find NEW ticker ideas (e.g., "undervalued AI stocks 2025").
    
    Args:
        query (str): Search query (e.g., "best biotech stocks under $50").
        
    Returns:
        dict: Top 5 search results with titles, snippets, and URLs.
    """
    try:
        results = []
        # specific 'news' backend is often good for markets, but 'text' is broader
        with DDGS() as ddgs:
            # Fetch top 5 results
            ddg_results = list(ddgs.text(query, max_results=5))
            
            if not ddg_results:
                return {"status": "error", "message": "No results found."}
            
            for r in ddg_results:
                results.append({
                    "title": r.get('title'),
                    "snippet": r.get('body'), # DDGS uses 'body' for snippet
                    "link": r.get('href')
                })
                
        return {
            "status": "success",
            "query": query,
            "results": results
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [10]:
# --- Test ---
print("Searching for Trending Stocks...")
print(search_market_trends("fastest growing AI stocks 2025"))

Searching for Trending Stocks...
{'status': 'success', 'query': 'fastest growing AI stocks 2025', 'results': [{'title': "AI Stocks : Best Artificial Intelligence Stocks ... | Investor's Business Dai...", 'snippet': 'AI Stocks : Snowflake Software Leader. Having struggled to generate new revenue from "copilots," software companies are now turning to autonomous, goal-driven AI agents. One big issue for software companies is how fast customers ramp up pilot programs to commercial deployment.', 'link': 'https://www.investors.com/news/technology/artificial-intelligence-stocks/'}, {'title': 'Prediction: 2 AI Stocks That Will Be Worth More Than... | Nasdaq', 'snippet': "That incredible acceleration makes it one of the market's fastest - growing AI stocks . But with an enterprise value of $1.07 billion, it still trades at less than 5 times its 2025 sales.", 'link': 'https://www.nasdaq.com/articles/prediction-2-ai-stocks-will-be-worth-more-bigbearai-2-years-now'}, {'title': 'Three AI stocks poi

## Step 2: Initialize Agent with Tools
We define the `tools` list containing our three functions and configure the Gemini Pro model to use them. We then start a chat session (Agent) that knows about these tools.


In [11]:
# 1. Define the Tool List
# We pass the actual function objects to the GenAI SDK. 
# The SDK automatically inspects the docstrings to understand how to use them.
my_tools = [
    get_asset_price_history,
    get_asset_fundamentals,
    calculate_technical_indicators
]

# 2. Initialize the Model with Tools
# We use 'gemini-1.5-flash' or 'gemini-1.5-pro' for best tool performance.
# 'auto' function calling mode is default, meaning the model decides when to use tools.
model = genai.GenerativeModel(
    model_name='gemini-2.5-flash', # Or 'gemini-1.5-pro' if you have access
    tools=my_tools
)

# 3. Create the Agent (Chat Session)
# enable_automatic_function_calling=True means the SDK handles the loop:
# Model asks for tool -> SDK runs tool -> SDK gives answer back to Model -> Model answers user.
agent = model.start_chat(enable_automatic_function_calling=True)



In [12]:
print("Agent initialized with Financial Tools!")

# --- Quick Test ---
# We ask a question that REQUIRES data to answer.
print("\nAsking Agent: 'What is the current volatility of Apple and is it in an uptrend?'")
response = agent.send_message("What is the current volatility of Apple (AAPL) and is it in an uptrend?")

print("\nAgent Answer:")
print(response.text)

Agent initialized with Financial Tools!

Asking Agent: 'What is the current volatility of Apple and is it in an uptrend?'

Agent Answer:
The current annualized volatility of Apple (AAPL) is 22.85%. Yes, it is in an uptrend as its current price (278.85) is above its 50-day Simple Moving Average (262.46).


## Step 3: Defining the Financial Advisor Persona
We re-initialize the agent with a **System Instruction**. This tells the model to act as a professional Investment Advisor, mandates that it *must* use tools before answering, and defines risk rules (e.g., "High volatility > 30% requires a warning").


In [13]:
# Define the System Prompt (Persona & Rules)
advisor_instruction = advisor_instruction = """
You are an expert Financial Research Assistant and Investment Advisor.

YOUR GOAL:
Help the user analyze assets and construct portfolios based on data.

RULES:
1. ALWAYS fetch real data using your tools before answering. Do not guess.
2. If a user asks about a stock, get its Fundamentals AND Technicals.
3. RISK ASSESSMENT:
   - If Annualized Volatility is > 30%, label the asset as "High Risk".
   - If Volatility is < 15%, label it as "Stable/Low Risk".
4. RECOMMENDATIONS: Cite specific metrics to support your view.

FORMATTING REQUIREMENTS:
- You MUST present data in two distinct tables:
  
  **Table 1: Fundamentals**
  | Metric | Value |


  **Table 2: Technicals**
  | Metric | Value |

- Make sure the table is well format in a uniform spaced manner with its borders
- Do not leave tables empty. If data is missing, write "N/A".
"""

# Re-initialize model with the system instruction
advisor_model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',
    tools=my_tools,
    system_instruction=advisor_instruction
)

# Start the Advisor Chat
advisor_agent = advisor_model.start_chat(enable_automatic_function_calling=True)

print("Advisor Agent initialized with System Instructions!")


Advisor Agent initialized with System Instructions!


In [14]:
# --- Test the Persona ---
query = "Analyze Tesla (TSLA). Is it safe for a conservative retiree?"
print(f"\nUser: {query}")
response = advisor_agent.send_message(query)

print("\nAdvisor Response:")
print(response.text)


User: Analyze Tesla (TSLA). Is it safe for a conservative retiree?

Advisor Response:
Here's an analysis of Tesla (TSLA):

**Table 1: Fundamentals**
| Metric             | Value               |
|:-------------------|:--------------------|
| Name               | Tesla, Inc.         |
| Symbol             | TSLA                |
| Sector             | Consumer Cyclical   |
| Industry           | Auto Manufacturers  |
| Market Cap         | 1.43 Trillion       |
| Trailing P/E       | 294.64              |
| Forward P/E        | 132.77              |
| Dividend Yield     | N/A                 |
| 52 Week High       | 488.54              |
| 52 Week Low        | 214.25              |
| Summary            | Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offer

## Step 4: Portfolio Allocation Tool
Defining `allocate_portfolio` to simulate investment execution. This tool takes a list of assets and a budget, fetches real-time prices, and calculates an equal-weighted allocation (share counts and values).


In [15]:
def allocate_portfolio(tickers: list[str], total_amount: float, weights: Dict[str, float] = None) -> Dict[str, Any]:
    """
    Create a portfolio allocation plan.
    
    Args:
        tickers (list[str]): List of symbols (e.g., ['AAPL', 'BTC-USD']).
        total_amount (float): Total cash to invest (e.g., 10000.0).
        weights (dict, optional): Dictionary of symbol->float (e.g., {'AAPL': 0.7, 'BTC-USD': 0.3}).
                                  Sum should be roughly 1.0. If None, defaults to Equal Weight.
        
    Returns:
        dict: Allocation plan including shares, values, and remaining cash.
    """
    try:
        if not tickers or total_amount <= 0:
            return {"status": "error", "message": "Invalid tickers or amount."}
        
        # 1. Determine Strategy (Custom vs Equal)
        strategy_name = "Custom Risk-Adjusted"
        if not weights:
            strategy_name = "Equal Weight"
            # Create equal weights: 1/N
            w_val = 1.0 / len(tickers)
            weights = {t: w_val for t in tickers}
        
        # Check weight sum roughly equals 1.0
        total_weight = sum(weights.values())
        if not (0.95 <= total_weight <= 1.05):
             return {"status": "error", "message": f"Weights must sum to 1.0 (got {total_weight})"}

        allocation = []
        total_spent = 0.0
        
        # 2. Execute Allocation Loop
        for symbol in tickers:
            # Basic heuristic for crypto symbols (if user forgot -USD)
            search_symbol = symbol
            if symbol in ["BTC", "ETH"] or (symbol.isupper() and len(symbol) <= 4 and "USD" not in symbol and symbol not in ["AAPL", "MSFT", "TSLA", "NVDA", "GOOG", "AMZN"]): 
                 pass # In a real app, we might auto-append -USD here
            
            # Get Current Price efficiently
            ticker_obj = yf.Ticker(search_symbol)
            try:
                price = ticker_obj.fast_info['last_price']
            except:
                # Fallback to history if fast_info fails
                hist = ticker_obj.history(period="1d")
                if not hist.empty:
                    price = hist['Close'].iloc[-1]
                else:
                    price = 0.0
            
            if price > 0:
                # Determine budget for this asset
                # Defaults to 0.0 if symbol not in weights dict (safety)
                asset_weight = weights.get(symbol, 0.0)
                asset_budget = total_amount * asset_weight
                
                shares = round(asset_budget / price, 4)
                cost = round(shares * price, 2)
                
                allocation.append({
                    "symbol": symbol,
                    "price": round(price, 2),
                    "shares": shares,
                    "value": cost,
                    "weight_allocated": f"{round((cost / total_amount) * 100, 1)}%"
                })
                total_spent += cost
            else:
                # If price is 0 or failed
                allocation.append({
                    "symbol": symbol,
                    "price": 0,
                    "error": "Could not fetch price"
                })
            
        remaining_cash = round(total_amount - total_spent, 2)
        
        return {
            "status": "success",
            "strategy": strategy_name,
            "total_investment": total_amount,
            "allocation": allocation,
            "total_spent": round(total_spent, 2),
            "remaining_cash": remaining_cash
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}

# --- Test ---
print("Testing Portfolio Tool...")
print(allocate_portfolio(["AAPL", "MSFT"], 1000, weights={"AAPL": 0.7, "MSFT": 0.3}))


Testing Portfolio Tool...
{'status': 'success', 'strategy': 'Custom Risk-Adjusted', 'total_investment': 1000, 'allocation': [{'symbol': 'AAPL', 'price': 278.85, 'shares': 2.5103, 'value': 700.0, 'weight_allocated': '70.0%'}, {'symbol': 'MSFT', 'price': 492.01, 'shares': 0.6097, 'value': 299.98, 'weight_allocated': '30.0%'}], 'total_spent': 999.98, 'remaining_cash': 0.02}


In [16]:
# --- Test ---
print("Testing Portfolio Tool...")
print(allocate_portfolio(["AAPL", "MSFT"], 1000, weights={"AAPL": 0.7, "MSFT": 0.3}))

Testing Portfolio Tool...
{'status': 'success', 'strategy': 'Custom Risk-Adjusted', 'total_investment': 1000, 'allocation': [{'symbol': 'AAPL', 'price': 278.85, 'shares': 2.5103, 'value': 700.0, 'weight_allocated': '70.0%'}, {'symbol': 'MSFT', 'price': 492.01, 'shares': 0.6097, 'value': 299.98, 'weight_allocated': '30.0%'}], 'total_spent': 999.98, 'remaining_cash': 0.02}


## Step 5: Final Agent Assembly
Updating the agent with the `allocate_portfolio` tool and refined instructions. The agent can now Research, Analyze Risk, AND Construct Portfolios in a single conversation.


In [17]:
# --- FINAL AGENT SETUP (Production Ready) ---

# 1. Consolidate all tools
final_tools = [
    get_asset_price_history,        # Tool 1.1
    get_asset_fundamentals,         # Tool 1.2
    calculate_technical_indicators, # Tool 1.3
    allocate_portfolio,             # Tool 1.4 (Portfolio)
    search_market_trends            # Tool 1.5 (Web Search)
]

# 2. robust System Instruction
final_instruction = """
You are a Strategic Investment Advisor (Powered by Gemini 2.5).

YOUR GOAL:
Build a solid core portfolio AND identify high-potential "Sleeper" stocks.

WORKFLOW:
1. **Discovery**: Search for trending/underrated ideas (`search_market_trends`).
2. **Verification**: Check data for ALL candidates (`get_asset_fundamentals`, `calculate_technical_indicators`).
3. **Selection**:
   - Select the BEST 3-5 assets for the **Main Portfolio** (Balance of Risk/Reward).
   - Select 1-2 assets that are interesting but didn't make the main cut (e.g., higher risk, smaller cap, turnaround play) for the **"Underrated"** list.

OUTPUT SECTIONS (Strictly follow this order):
1. **Market Insight**: "Based on search trends, I focused on [Sector]..."
2. **Core Portfolio Analysis**:
   - Data Tables (Fundamentals & Technicals) for the main picks with spaced formating for better readability.
   - Brief descrition on the choicess made.
3. **Proposed Allocation**: The `allocate_portfolio` table.
4. **💎 Underrated Stocks to Consider**:
   - List 1-2 stocks that are NOT in the main portfolio above.
   - **Why?**: Explain the "Growth Story" vs "Why it was excluded" (e.g., "High potential due to AI news, but excluded from core due to 85% volatility. Good for a small speculative bet.").
5. **Glossary**: Define technical terms.

DISCLAIMER: "This is a simulation for educational purposes."
"""

# Initialize Model with the specific version
# Ensure your SDK version supports this model name string.
final_model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',  # <--- UPDATED HERE
    tools=final_tools,
    system_instruction=final_instruction
)



In [18]:
# 4. Start Session
analyst_desk = final_model.start_chat(enable_automatic_function_calling=True)

print("Final 'Trend Hunter' Analyst Desk is Ready!")
print("Type 'exit' to stop.\n")

# --- INTERACTIVE LOOP ---
while True:
    user_input = input("👤 You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    
    print("Agent is thinking...")
    try:
        response = analyst_desk.send_message(user_input)
        print(f"\nAnalyst:\n{response.text}\n")
        print("-" * 80)
    except Exception as e:
        print(f"Error: {e}")

Final 'Trend Hunter' Analyst Desk is Ready!
Type 'exit' to stop.

Agent is thinking...

Analyst:
| Symbol | Shares   | Value ($) | Weight Allocated (%) |
|--------|----------|-----------|----------------------|
| AAPL   | 12.55    | 3500.01   | 35.0                 |
| MSFT   | 7.11     | 3500.01   | 35.0                 |
| BTC-USD| 0.033    | 3001.01   | 30.0                 |

💎 Underrated Stocks to Consider:

**NVIDIA (NVDA)**

**Why?**: NVIDIA is a leader in graphics processing units (GPUs) and a key enabler of artificial intelligence and high-performance computing. Its growth story is driven by the increasing demand for AI, data centers, and gaming. The company has a strong innovation pipeline and is well-positioned for future technological advancements.

**Why it was excluded from the core portfolio?**: While NVIDIA presents a compelling growth story, its annualized volatility of 37.41% is considerably higher than Apple and Microsoft. Additionally, it is currently in a downtrend